# A Movie behind a Script


In [1]:
import os
import re
import findspark
import pandas as pd
findspark.init()
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import urllib.request
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.10:0.4.1 pyspark-shell'

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'UTC')
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Overview of datasets

The OpenSubtitles dataset is a compressed cluster of folders containing XML files. Each XML file is split into a script portion with the subtitles of the movie and a metadata portion with additional information about the movie or show. The name of one of the parent folders of the XML file is the corresponding IMDb identifier of the movie or show, thus allowing us to extract additional information from the IMDb dataset.

## IMDb Dataset

We have at our disposal the IMDb ratings and basics dataset. For the moment we have downloaded the files locally, but we would like to scrape the data.

In [3]:
# TODO scrape data https://datasets.imdbws.com/
ratings_fn = "title.ratings.tsv.gz"
basics_fn = "title.basics.tsv.gz"

In [4]:
df_ratings = spark.read.option("header", "true")\
                       .option("sep", "\t")\
                       .csv("imdb_data/" + ratings_fn)
df_ratings.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.8|    1440|
|tt0000002|          6.3|     172|
|tt0000003|          6.6|    1041|
|tt0000004|          6.4|     102|
|tt0000005|          6.2|    1735|
|tt0000006|          5.5|      91|
|tt0000007|          5.5|     579|
|tt0000008|          5.6|    1539|
|tt0000009|          5.6|      74|
|tt0000010|          6.9|    5127|
|tt0000011|          5.4|     214|
|tt0000012|          7.4|    8599|
|tt0000013|          5.7|    1318|
|tt0000014|          7.2|    3739|
|tt0000015|          6.2|     660|
|tt0000016|          5.9|     982|
|tt0000017|          4.8|     197|
|tt0000018|          5.5|     414|
|tt0000019|          6.6|      13|
|tt0000020|          5.1|     232|
+---------+-------------+--------+
only showing top 20 rows



In [5]:
df_basics = spark.read.option("header", "true")\
                      .option("sep", "\t")\
                      .csv("imdb_data/" + basics_fn)
df_basics.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            \N|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

## OpenSubtitles dataset

The dataset consists of 31 GB of XML files distributed in the following file structure: 

```
├── opensubtitle
│   ├── OpenSubtitles2018
│   │   ├── Year
│   │   │   ├── Id
│   │   │   │   ├── #######.xml.gz
│   │   │   │   ├── #######.xml.gz
│   ├── en.tar.gz
│   ├── fr.tar.gz
│   ├── zh_cn.tar.gz
```
where
- `######` is a 6-digit unique identifier of the file on the OpenSubtitles dataset.
- `Year` is the year the movie or episode was made.
- `Id` is a 5 to 7 digit identifier (if it's 7-digit it's also an IMDb identifier).

The subtitles are provided in different languages. We only analyze the `OpenSubtitles2018` folder and it's the only folder we detail.

The decompressed XML files vary in size, ranging from 5KB to 9000KB sized files.

## XML Files

Each XML file is split into a `document` and `metadata` section.

### Subtitles

The `document` section contains all the subtitles and its general structure is the following:

```
├── s
│   ├── time: Integer
│   ├── w: String
```

An example snippet of an XML file:

```xml
  <s id="1">
    <time id="T1S" value="00:00:51,819" />
    <w id="1.1">Travis</w>
    <w id="1.2">.</w>
    <time id="T1E" value="00:00:53,352" />
  </s>
```

The subtitles in each XML file are stored by blocks denoted by `s` with a unique `id` attribute (integers in increasing order starting at 1).  

Each block (`<s id="1">` for instance) has a:  

1. Set of timestamps (denoted by `time`) with
 - A timestamp `id` attribute that can take two different formats: `T#S` or `T#E`, where _S_ indicates _start_, _E_ indicates _end_ and _#_ is an increasing integer. 
 - A `value` attribute which has the format `HH:mm:ss,fff`.

2. Set of words (denoted by `w`) with
 - an `id` attribute that is simply an increasing number of decimal numbers of the format `X.Y` where X is the string id and Y is the word id within the corresponding string
 - a non-empty `value` attribute that contains a token: a word or a punctuation character. 

It sometimes also has an `alternative`, `initial` and `emphasis` attribute.  

 - The `initial` attribute generally corresponds to slang words or mispronounced words because of an accent such as _lyin'_ instead of _lying_.  
 - The `alternative` attribute is another way of displaying the subtitle for example _HOW_ instead of _how_.
 - The `emphasis` attribute is a boolean.

### Metadata

The `metadata` section has the following structure:

```
├── Conversion
│   ├── corrected_words: Integer
│   ├── sentences: Integer
│   ├── tokens: Integer
│   ├── encoding: String (always utf-8)
│   ├── unknown_words: Integer
│   ├── ignored_blocks: Integer
│   ├── truecased_words: Integer
├── Subtitle
│   ├── language: String
│   ├── date: String
│   ├── duration: String
│   ├── cds: String (presented as #/# where # is an int)
│   ├── blocks: Integer
│   ├── confidence: Double
├── Source
│   ├── genre: String[] (up to 3 genres)
│   ├── year: Integer
│   ├── duration: Integer (in minutes)
│   ├── original: String
│   ├── country: String
```

We note that some XML files may not have all the entries. 
We can use the metadata to obtain additional information about the movie or show's subtitles and compute certain statistics. 

## 

In [5]:
df_sample_document = sqlContext.read.format('com.databricks.spark.xml')\
                                    .options(rowTag='document') \
                                    .load('sample_dataset/2017/6464116/6887453.xml.gz')

In [6]:
df_sample_metadata = sqlContext.read.format('com.databricks.spark.xml')\
                                    .options(rowTag='meta') \
                                    .load('sample_dataset/2017/6464116/6887453.xml.gz')

df_sample_metadata.printSchema()
df_sample_metadata.show()

root
 |-- conversion: struct (nullable = true)
 |    |-- corrected_words: long (nullable = true)
 |    |-- encoding: string (nullable = true)
 |    |-- ignored_blocks: long (nullable = true)
 |    |-- sentences: long (nullable = true)
 |    |-- tokens: long (nullable = true)
 |    |-- truecased_words: long (nullable = true)
 |    |-- unknown_words: long (nullable = true)
 |-- source: struct (nullable = true)
 |    |-- duration: long (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- year: long (nullable = true)
 |-- subtitle: struct (nullable = true)
 |    |-- blocks: long (nullable = true)
 |    |-- cds: string (nullable = true)
 |    |-- confidence: double (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- duration: string (nullable = true)
 |    |-- language: string (nullable = true)

+--------------------+--------------------+--------------------+
|          conversion|              source|            subtitle|
+--------------------+------------

Here we can see the schema. We need to decide what is actually relevant for us to filter out the useless information and choose which format our dataframe should have (for example having all the different genres in a separate column.)

For the metadata we have a very clean dataframe which can be used for a lot of statistics and filtering : filtering by genre, by year etc. 

We can see that there is no actual link between our dataframes (subtitles and metadata) : neither contain the movie id that would pair them. 

## Exploration

Going through the dataset we notice a few things:

1. The dataset has meaningless folders. For example, the folder 1858/ is empty.
2. Dataset contains XML files that are not related to movies or TV shows. For example, the folder 666/ contains Justin Bieber song subtitles.  
3. Trailer of films can be present in the dataset. For example, the folder 2018/ we found for example Black Panther teaser trailer subtitles.
4. Each movie might have more than 1 subtitle file.
5. Some subtitle files contain text that is not related to the movie, like credits to the person who made the subtitles.
6. The IDMDb folder name is not always a 7-digit number, meaning it is not always a valid IMDb identifer and we can't retrieve the IMDb info.
7. Each block may have an arbitrary number (including 0) of timestamps associated to it.

To solve points 1 and 2, we ignore all the folders which aren't inside the range of 1920-2018.

To solve point 3, we drop trailers by looking at the `duration` field in the metadata section.

To solve point 4, we simply take the first one.

To solve point 6, we keep movies that have a correct IMDb identifier. Hence, all the files in folders that don't have a 7-digit folder name are dropped.

To solve point 7, we decide not to associate a timestamp to each word for the moment.
 
For the moment, we take a sample of the dataset from the cluster (see python script `extract_sample_2.py`) by collecting 1 or 2 movies for each year in the range 1920-2018.

## Cleaning and structuring data

### Subtitles 

In [7]:
df_sample_film = sqlContext.read.format('com.databricks.spark.xml')\
                                .options(rowTag='s') \
                                .load('sample_dataset/2017/6464116/6887453.xml.gz')
imdb_id = '6464116'
df_sample_film.printSchema()
df_sample_film.show()

root
 |-- _emphasis: boolean (nullable = true)
 |-- _id: long (nullable = true)
 |-- time: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- w: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _alternative: string (nullable = true)
 |    |    |-- _emphasis: boolean (nullable = true)
 |    |    |-- _id: double (nullable = true)

+---------+---+--------------------+--------------------+
|_emphasis|_id|                time|                   w|
+---------+---+--------------------+--------------------+
|     null|  1|[[, T1S, 00:00:01...|[[[,,, 1.1], [sho...|
|     null|  2|[[, T2S, 00:00:09...|[[[,,, 2.1], [buz...|
|     true|  3|[[, T4S, 00:00:12...|[[Clear,,, 3.1], ...|
|     null|  4|[[, T5S, 00:00:14...|[[When,,, 4.1], [...|
|     true

We have the schema and the first 20 entries of the dataframe containing the subtitles. We see that it contains a lot of null values and information we want to get rid of. Each word array contains an Id we don't need and per row entry we have an array of arrays with words and the times.

We consider storing the sentences in a list of words as it seems to be the best way to perform queries such as counting the number of distinct words or counting the common words between films. We create a second function `udf_sentence` to generate the sentence as a single string.

In [8]:
def to_sentence(words):
    """Function to map the struct containing the words 
    to a list of words """
    w_list = []
    for w in words:
        if w['_VALUE']:
            w_list.append(w['_VALUE'])
    return w_list

In [9]:
# Transform to spark function
udf_word_array = udf(to_sentence, ArrayType(StringType()))
# Define function to create sentence 
udf_sentence = udf(lambda x: ' '.join([w[0] for w in x]), StringType())

In [10]:
# Compute sentences (list of words in order)
df_sample_film_sentence_list = df_sample_film.withColumn("sentence", udf_word_array("w"))

In [11]:
df_sample_film_sentence_list.show()

+---------+---+--------------------+--------------------+--------------------+
|_emphasis|_id|                time|                   w|            sentence|
+---------+---+--------------------+--------------------+--------------------+
|     null|  1|[[, T1S, 00:00:01...|[[[,,, 1.1], [sho...|[[, shots, firing...|
|     null|  2|[[, T2S, 00:00:09...|[[[,,, 2.1], [buz...|[[, buzzer, ], [,...|
|     true|  3|[[, T4S, 00:00:12...|[[Clear,,, 3.1], ...|[Clear, and, hols...|
|     null|  4|[[, T5S, 00:00:14...|[[When,,, 4.1], [...|[When, your, weap...|
|     true|  5|[[, T6S, 00:00:18...|[[The,,, 5.1], [l...|[The, line, is, s...|
|     true|  6|[[, T6E, 00:00:21...|[[Remove,,, 6.1],...|[Remove, your, ey...|
|     null|  7|[[, T7S, 00:00:21...|[[Bring,,, 7.1], ...|[Bring, your, tar...|
|     null|  8|[[, T7E, 00:00:24...|[[Watch,,, 8.1], ...|[Watch, your, hea...|
|     null|  9|[[, T8S, 00:00:27...|[[Police,,, 9.1],...|[Police, your, br...|
|     null| 10|[[, T9S, 00:00:29...|[[Bring,,, 10.1]

In [12]:
df_sample_film_sentence_string = df_sample_film.withColumn("sentence", udf_sentence("w"))
df_sample_film_sentence_string.show()

+---------+---+--------------------+--------------------+--------------------+
|_emphasis|_id|                time|                   w|            sentence|
+---------+---+--------------------+--------------------+--------------------+
|     null|  1|[[, T1S, 00:00:01...|[[[,,, 1.1], [sho...|    [ shots firing ]|
|     null|  2|[[, T2S, 00:00:09...|[[[,,, 2.1], [buz...|[ buzzer ] [ casi...|
|     true|  3|[[, T4S, 00:00:12...|[[Clear,,, 3.1], ...|Clear and holster...|
|     null|  4|[[, T5S, 00:00:14...|[[When,,, 4.1], [...|When your weapon ...|
|     true|  5|[[, T6S, 00:00:18...|[[The,,, 5.1], [l...|  The line is safe .|
|     true|  6|[[, T6E, 00:00:21...|[[Remove,,, 6.1],...|Remove your eyes ...|
|     null|  7|[[, T7S, 00:00:21...|[[Bring,,, 7.1], ...|Bring your target...|
|     null|  8|[[, T7E, 00:00:24...|[[Watch,,, 8.1], ...|  Watch your heads .|
|     null|  9|[[, T8S, 00:00:27...|[[Police,,, 9.1],...| Police your brass .|
|     null| 10|[[, T9S, 00:00:29...|[[Bring,,, 10.1]

In [13]:
# Explode sentences to words for word counts
df_sample_film_words = df_sample_film_sentence_list.select('*', explode(col("sentence")).alias('word'))
# Filter strings that are not words
df_sample_film_words = df_sample_film_words.filter(df_sample_film_words.word.rlike("^[a-zA-Z]+$"))

In [14]:
word_count_distinct = df_sample_film_words.select("word").distinct().count()
word_count_total = df_sample_film_words.select("word").count()

print("Number of distinct words in film is: {:}".format(word_count_distinct))
print("Total number of  words in film is: {:}".format(word_count_total))

Number of distinct words in film is: 1341
Total number of  words in film is: 5454


In [15]:
def film_words(df_film):
    """Function that returns proper dataframe and word statistics"""
    df_words = df_film.withColumn("sentence", udf_word_array("w")) \
                      .select('*', explode(col("sentence")).alias('word'))
    
    #TODO change udf_sentence to filter out empty strings and marks.
    
    # Filter punctuation characters 
    df_words_filter = df_words.filter(df_words.word.rlike("^[a-zA-Z]+$"))
    word_count_distinct = df_words_filter.select("word").distinct().count()
    word_count_total = df_words_filter.select("word").count()
    
    return df_words_filter, word_count_distinct, word_count_total

In [16]:
def subtitle_df(df_film, identifier):
    """Function that returns proper dataframe and word statistics"""
    df_words, word_count, total_words = film_words(df_film)
    
    df_result = df_words.withColumn("imdb_id", lit(identifier))\
                        .selectExpr("imdb_id", "_id as sentence_id", "word")
    
    return df_result, word_count, word_count_total

In [17]:
film, words_distinct, words_total = subtitle_df(df_sample_film, imdb_id)

print('Number of distinct words : {}'.format(words_distinct))
print('Number of words : {}'.format(words_total))
film.show()

Number of distinct words : 1341
Number of words : 5454
+-------+-----------+-------+
|imdb_id|sentence_id|   word|
+-------+-----------+-------+
|6464116|          1|  shots|
|6464116|          1| firing|
|6464116|          2| buzzer|
|6464116|          2|casings|
|6464116|          2|clatter|
|6464116|          3|  Clear|
|6464116|          3|    and|
|6464116|          3|holster|
|6464116|          3|   your|
|6464116|          3|   safe|
|6464116|          3| weapon|
|6464116|          4|   When|
|6464116|          4|   your|
|6464116|          4| weapon|
|6464116|          4|     is|
|6464116|          4|   safe|
|6464116|          4|  raise|
|6464116|          4|   your|
|6464116|          4|   hand|
|6464116|          5|    The|
+-------+-----------+-------+
only showing top 20 rows



In [18]:
#Function for clean processing
#TODO consider if "explode" beforehand or not
def subtitle_cleaning(df_film, identifier):
    df_words = df_film.withColumn("sentence", udf_word_array("w")) \
                      .select('*', explode(col("sentence")).alias('word'))
    df_words_filter = df_words.filter(df_words.word.rlike("^[a-zA-Z]+$"))
    df_result = df_words_filter.withColumn("imdb_id", lit(identifier)).select("imdb_id", "_id", "word")
    return df_result

### Metadata

We create a function which will give us a resulting metadata dataframe with the information we want. We separate genres as an array of strings for later queries. We also associate an IMDb Id.

In [19]:
#TODO check if convert genres to lowercase?
#TODO what to do if missing columns: source duration, need to choose

udf_split = udf(str.split, ArrayType(StringType()))

def meta_data_cleaning(df_metadata, identifier):
    df = df_metadata.withColumn("imdb_id", lit(identifier)).selectExpr("imdb_id", 
                                                                       "conversion.sentences",
                                                                       "source.genre", 
                                                                       "source.year", 
                                                                       "subtitle.blocks",
                                                                       "subtitle.duration as subtitle_duration",
                                                                       "subtitle.language")
    df = df.withColumn("genres", udf_split("genre")).drop("genre")
    return df

df_md = meta_data_cleaning(df_sample_metadata, imdb_id)

In [20]:
df_md.show()

+-------+---------+----+------+-----------------+--------+--------------------+
|imdb_id|sentences|year|blocks|subtitle_duration|language|              genres|
+-------+---------+----+------+-----------------+--------+--------------------+
|6464116|      967|2017|   888|     00:40:42,361| English|[Action,Crime,Drama]|
+-------+---------+----+------+-----------------+--------+--------------------+



## To be classified

After processing a simple XML file, we code a program to make a resulting dataframe where we can make different queries.

In [21]:
def dataframe_maker(path):
    df_md = sqlContext.read.format('com.databricks.spark.xml')\
                                    .options(rowTag='meta') \
                                    .load(path)
    df_sub = sqlContext.read.format('com.databricks.spark.xml')\
                                    .options(rowTag='s') \
                                    .load(path)
    return (df_md, df_sub)

In [22]:
path = "sample_dataset/"
df_metadatas = spark.createDataFrame([], df_md.schema)
df_subtitles = spark.createDataFrame([], film.schema)
for year in os.listdir(path):
    for imdb_id in os.listdir(path + year):
        current_path = path + year + "/" + imdb_id
        for file in os.listdir(current_path):
            (df_m, df_sub) = dataframe_maker(current_path + '/' + file)
            df_metadatas = df_metadatas.union(meta_data_cleaning(df_m, imdb_id))
            df_subtitles = df_subtitles.union(subtitle_cleaning(df_sub, imdb_id))
#             df_m.show()
#             print(current_path + "/" + file)

We now have a metadata and subtitle dataframe

In [23]:
df_metadatas.show()

+-------+---------+----+------+-----------------+--------+--------------------+
|imdb_id|sentences|year|blocks|subtitle_duration|language|              genres|
+-------+---------+----+------+-----------------+--------+--------------------+
|1165285|      337|1924|   319|     00:28:04,132| English|             [Short]|
|1452522|      208|1924|   181|     00:53:20,553| English| [Drama,History,War]|
|1002599|       10|1927|     9|     00:03:30,217| English|   [Animation,Short]|
|1320310|       49|1928|    47|     00:07:33,581| English|      [Comedy,Short]|
|1886619|       98|1928|    76|     00:54:09,179| English|             [Drama]|
|1002784|      141|1934|   138|     00:55:59,125| English|             [Drama]|
|1002784|      138|1934|   138|     00:55:58,777| English|             [Drama]|
|1703934|       64|1935|    69|     00:09:51,450| English|   [Animation,Short]|
|1855316|      578|1936|   555|     01:23:40,678| English|               [War]|
|2082346|      343|1936|   575|     00:4

In [24]:
df_subtitles.show()

+-------+-----------+-----------+
|imdb_id|sentence_id|       word|
+-------+-----------+-----------+
|1165285|          1|   BACKWARD|
|1165285|          1|    CURRENT|
|1165285|          2|   Original|
|1165285|          2|      Story|
|1165285|          2| Adaptation|
|1165285|          3|    Rokuhei|
|1165285|          3|   Susukita|
|1165285|          3|   Director|
|1165285|          4|    Buntaro|
|1165285|          4|   Futagawa|
|1165285|          5|Tsumasaburo|
|1165285|          5|      Bando|
|1165285|          5|         as|
|1165285|          5| Mikisaburo|
|1165285|          5|      Nanjo|
|1165285|          5|     Kotaro|
|1165285|          5|    Kataoka|
|1165285|          5|         as|
|1165285|          5|  Genzaburo|
|1165285|          5|     Hayami|
+-------+-----------+-----------+
only showing top 20 rows



In [25]:
df_subtitles = df_subtitles.groupBy("imdb_id").count()
df_subtitles

DataFrame[imdb_id: string, count: bigint]

In [26]:
spark.conf.set("spark.sql.crossJoin.enabled", "true")
df_md.join(film, ["imdb_id"]).show()

+-------+---------+----+------+-----------------+--------+--------------------+-----------+-------+
|imdb_id|sentences|year|blocks|subtitle_duration|language|              genres|sentence_id|   word|
+-------+---------+----+------+-----------------+--------+--------------------+-----------+-------+
|6464116|      967|2017|   888|     00:40:42,361| English|[Action,Crime,Drama]|          1|  shots|
|6464116|      967|2017|   888|     00:40:42,361| English|[Action,Crime,Drama]|          1| firing|
|6464116|      967|2017|   888|     00:40:42,361| English|[Action,Crime,Drama]|          2| buzzer|
|6464116|      967|2017|   888|     00:40:42,361| English|[Action,Crime,Drama]|          2|casings|
|6464116|      967|2017|   888|     00:40:42,361| English|[Action,Crime,Drama]|          2|clatter|
|6464116|      967|2017|   888|     00:40:42,361| English|[Action,Crime,Drama]|          3|  Clear|
|6464116|      967|2017|   888|     00:40:42,361| English|[Action,Crime,Drama]|          3|    and|


## Reflection (updated README too)

_That you considered ways to enrich, filter, transform the data according to your needs._

We managed to locally download the IMDb dataset and load it into a dataframe but we would like to scrape the datasets. On top of this, we would also like to scrape some more data from the IMDb website not present in the IMDb datasets: box office and reviewers' comments.

_That you have updated your plan in a reasonable way, reflecting your improved knowledge after data acquaintance. In particular, discuss how your data suits your project needs and discuss the methods you’re going to use, giving their essential mathematical details in the notebook._

We can easily compute word statistics (such as mean length of words, number of distinct words & total number of words) given that we successfully transformed the XML files into spark dataframes. That said, we still can not figure out why `df.groupBy("imdb_id").count()` does not work for us, and we will tackle this problem before continuing.

We will define sets of words for certain themes (e.g. love, hate, sex, drugs, war, slang...) and see how they correlate with the movie's popularity (IMDb rating, number of votes and popularity). In order to do that, we will define functions to query the text. We might create "super strings" that contain the whole subtitles of a movie and use those to see how many of words defined in the sets appear in the scripts.

We are also considering to use the SpaCy library in order to give a score to the similarity of differents scripts. This will help us in defining new categories of films. 

_That your plan for analysis and communication is now reasonable and sound, potentially discussing alternatives to your choices that you considered but dropped._

We realized that extracting the timestamp from each subtitle block was more complicated than expected. We thought we would find one start time and one end time in each block, but there was actually no consistency whatsoever. 

For the moment the plan seems ambitious but feasible.

The main difficulty will be to determine the features that are meaningful in order to group movies, implement a Machine Learning pipeline for the prediction and scrape IMDb's website for the reviewer's comments and missing figures (box-office).